In [1]:
import os
import re
import time
import json
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai.api_key = os.environ["OPENAI_API_KEY"]

- Summerize key points in the client meta data

In [2]:
example = {'description': 'Robert King has worked at various hedge funds. He is currently CEO at one in San Francisco. He serves on multiple local boards and is very wealthy.',
 'docs': {'_linkedin.html': {'Employment 1': 'Hedge Fund A, CEO',
   'Employment 2': 'Hedge Fund B, Senior Portfolio Manager',
   'Education': 'Stanford University, MBA',
   'Board Member': 'San Francisco Symphony, Board Director',
   'Bio': 'Robert King is an experienced finance professional with a strong background in hedge funds. He currently serves as the CEO of Hedge Fund A in San Francisco. Prior to this role, he worked as a Senior Portfolio Manager at Hedge Fund B. Robert is also actively involved in the community and serves as a Board Director for the San Francisco Symphony.'},
  '_wealthx.html': {'Estimated Net Worth': 'At least $100 million',
   'Estimated Liquid Assets': 'At least $50 million',
   'Estimated Household Wealth': 'At least $150 million',
   'Estimated Household Liquid Assets': 'At least $50 million',
   'Estimated Family Net Worth': 'At least $500 million',
   'Interests, Passions, Hobbies': 'Art collecting and philanthropy'},
  '_relsci.html': {'Boards & Committees (Corporate)': 'Tech Company X, Board Director',
   'Boards & Committees (Nonprofit)': 'San Francisco Museum of Modern Art, Trustee',
   'Former/Prior Boards & Committees (Corporate)': 'Investment Firm Y, Board Director',
   'Former/Prior Boards & Committees (Nonprofit)': 'Local Charity Z, Trustee',
   'Top donations (Nonprofit)': 'American Red Cross - $1M, Boys & Girls Clubs of America - $500K, Habitat for Humanity - $250K',
   'Top donations (Political parties)': 'Democratic National Committee - $100K, Joe Biden - $250K'},
  '_equilar.html': {'Stock sold - Equity Transactions (Last 36 Months)': '$50 million',
   'New Equity Grants - Equity Transactions (Last 36 Months)': '$20 million',
   'Options Exercised - Equity Transactions (Last 36 Months)': '$10 million',
   'Equity Holdings - Equity Transactions': '$80 million',
   'Annual Compensation': '$5 million',
   'Stock Sold': 'CEO, $1M at Hedge Fund A, June 1 2023 (SEC) | June 1 2023 (Effective)'},
  '_zoominfo.html': {'Personal Email': 'robertking@email.com'},
  '_pitchbook.html': {'Lead partner on deals': {'Company': 'Tech Startup XYZ',
    'Deal Date': 'May 15 2023',
    'Deal Type': 'Series B',
    'Deal Size': '$30M',
    'Deal Status': 'Completed',
    'Location': 'San Francisco, CA',
    'Representing': 'Hedge Fund A',
    'Other Partners': 'John Doe, Jane Smith, Mark Johnson'}},
  '_google.html': {'Article 1': {'Title': 'Robert King donates $1 million to local charity',
    'Date': 'March 10, 2023',
    'Abstract': 'In a generous act of philanthropy, Robert King has donated $1 million to a local charity. The donation will support various programs and initiatives aimed at improving the lives of underprivileged individuals in the community.'}},
}}

In [3]:
kwargs = {
    'max_tokens':1000,
    'verbose':False,
    'temperature':0,
    'top_p':1,
    'frequency_penalty':0,
    'presence_penalty':0
}

def get_completion_from_messages(messages, model="gpt-3.5-turbo", verbose=False, **kwargs):
    """
    Prime the model with one or more input prompts
        messages - list of prompts (each prompt is a dictionary of "role" and "content")
        model - ChatGPT model to use
        temperature - Single values in the range [0, 2]. Higher values
            produce more randomness.
        verbose - if True, print the full response and then
            return only the chat response
    """
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        **kwargs
    )
    if verbose:
        print(f"full response:\n{response}")
    return response.choices[0].message["content"]

In [4]:
# Output schema:
metadata_schema = {"_linkedin.html": {
                  "Employment 1": "Big Bank, Senior Data Scientist",
                  "Employment 2": "Small Bank, Data Scientist",
                  "Education": "UC Berkeley, PhD Astronomy",
                  "Board Member": "Girls and Boys Club, Chairman",
                  "Bio": "biography of up to 100 words that is consistent with the above information",
                 },

                 "_wealthx.html": {
                    "Estimated Net Worth": "At least $19.5 million",
                    "Estimated Liquid Assets": "At least $11.5 million",
                    "Estimated Household Wealth": "At least $31.5 million",
                    "Estimated Household Liquid Assets": "At least $11.5 million",
                    "Estimated Family Net Worth": "At least $226.1 billion",
                    "Interests, Passions, Hobbies" : "Tennis and golf",
                 },

                 "_relsci.html": {
                     "Boards & Committees (Corporate)": "Paramount Global, Board Director",
                     "Boards & Committees (Nonprofit)": "Brentwood School - California, Trustee",
                     "Former/Prior Boards & Committees (Corporate)": "The Walt Disney Company, Board Director",
                     "Former/Prior Boards & Committees (Nonprofit)": "The Paley Center for Media, Trustee",
                     "Top donations (Nonprofit)": "Greenpeace - $2M, UCSF - $1M, Kiva - $500K",
                     "Top donations (Political parties)": "Kamala Harris - $100K, CA Democratic Committee - $250K",
                 },

                 "_equilar.html": {
                    "Stock sold - Equity Transactions (Last 36 Months)": "$215.3 million",
                    "New Equity Grants - Equity Transactions (Last 36 Months)": "$93 million",
                    "Options Exercised - Equity Transactions (Last 36 Months)": "$17.5 million",
                    "Equity Holdings - Equity Transactions": "$167.5 million",
                    "Annual Compensation": "$11.9 million",
                    "Stock Sold": "CEO and Chairperson, $1.6M at Cerevel Therapeutics Holdings, Inc. (50,000 shares), June 5 2023 (SEC) | June 1 2023 (Effective)",
                 },

                 "_zoominfo.html": {
                     "Personal Email": "email@domain.com",
                 },

                 "_pitchbook.html": {
                     "Lead partner on deals": {
                          "Company": "Harvey (Business/Productivity Software)",
                          "Deal Date": "April 26 2023",
                          "Deal Type": "Early Stage VC",
                          "Deal Size": "$21M",
                          "Deal Status": "Completed",
                          "Location": "Los Angeles, CA",
                          "Representing": "Sequoia Capital",
                          "Other Partners": "Rich Dude I, Rich Guy II, Notso Rich III, Rich Wannabe",
                          #"Other Partners emails": "richdude@aristocracy.com, richguy@aristocracy.com",
                     },
#                     "Investor bio": "A brief summary of the investor, his peers, and his investment deals"
                 },

                 "_google.html": {
                     "Article 1": {
                         "Title": 'John Smith granted key to the city',
                         'Date': 'January 1, 2023',
                         "Abstract": '''In an expensive public ceremony, the mayor granted John Smith the key to the city. The mayor then spoke for 20 minutes on how great of a person is John Smith and how luck we are to be his contemporaries.'''
                         },
                 },
                 ## Should we add info on:
                     # criminal background
                     # if existing client, current relationship depth (e.g. CRB info)
                }

In [5]:
message_1_content = """***Client description: {client_description}
metadata: {client_metadata}*** \
<Your repsonse here> """

In [6]:
messages =  [  
{"role":"system",
 "content":f"""Given the following detailed meta data about an individual, \
generate a concise summary that captures the key details and any other significant \
aspects from the meta data and is consistent with the provided information:

 metadata: {metadata_schema}
 """},    
{"role":"user", "content": None},    
]

In [7]:
messages[1]['content'] = message_1_content.format(client_description=example['description'], client_metadata=example['docs'])

In [8]:
print(messages[1]['content'])

***Client description: Robert King has worked at various hedge funds. He is currently CEO at one in San Francisco. He serves on multiple local boards and is very wealthy.
metadata: {'_linkedin.html': {'Employment 1': 'Hedge Fund A, CEO', 'Employment 2': 'Hedge Fund B, Senior Portfolio Manager', 'Education': 'Stanford University, MBA', 'Board Member': 'San Francisco Symphony, Board Director', 'Bio': 'Robert King is an experienced finance professional with a strong background in hedge funds. He currently serves as the CEO of Hedge Fund A in San Francisco. Prior to this role, he worked as a Senior Portfolio Manager at Hedge Fund B. Robert is also actively involved in the community and serves as a Board Director for the San Francisco Symphony.'}, '_wealthx.html': {'Estimated Net Worth': 'At least $100 million', 'Estimated Liquid Assets': 'At least $50 million', 'Estimated Household Wealth': 'At least $150 million', 'Estimated Household Liquid Assets': 'At least $50 million', 'Estimated Fam

In [9]:
kwargs = {
    'verbose':False,
    'temperature':0,
    'top_p':1,
    'frequency_penalty':0,
    'presence_penalty':0
}

response = get_completion_from_messages(messages, model="gpt-3.5-turbo", **kwargs)

In [19]:
response

'Robert King is a highly accomplished finance professional with a strong background in hedge funds. He currently serves as the CEO of Hedge Fund A in San Francisco and previously worked as a Senior Portfolio Manager at Hedge Fund B. Robert is actively involved in the community and serves as a Board Director for the San Francisco Symphony. He is estimated to have a net worth of at least $100 million, with significant liquid assets. Robert is also engaged in philanthropy and has made substantial donations to organizations such as the American Red Cross and Boys & Girls Clubs of America. He holds board positions in both corporate and nonprofit entities, including Tech Company X and the San Francisco Museum of Modern Art. In addition, Robert has been involved in successful deals as a lead partner, representing Hedge Fund A in a recent Series B investment in Tech Startup XYZ.'

In [15]:
kwargs = {
    'max_tokens':200,
    'verbose':False,
    'temperature':0,
    'top_p':1,
    'frequency_penalty':0,
    'presence_penalty':0
}

response_limited_tokens = get_completion_from_messages(messages, model="gpt-3.5-turbo", **kwargs)

In [16]:
response_limited_tokens

'Robert King is a highly accomplished finance professional with a strong background in hedge funds. He currently serves as the CEO of Hedge Fund A in San Francisco and previously worked as a Senior Portfolio Manager at Hedge Fund B. Robert is actively involved in the community and serves as a Board Director for the San Francisco Symphony. He is estimated to have a net worth of at least $100 million, with significant liquid assets. Robert is also engaged in philanthropy and has made substantial donations to organizations such as the American Red Cross and Boys & Girls Clubs of America. He holds board positions in both corporate and nonprofit entities, including Tech Company X and the San Francisco Museum of Modern Art. In addition, Robert has been involved in successful deals as a lead partner, representing Hedge Fund A in a recent Series B investment in Tech Startup XYZ.'

- prompt template for transaction sturcture

In [24]:
categories = {
    "Earned Income": [
        "Freelance", 
        "Paycheck", 
        "Rideshare and delivery"
    ], 
    "Essential Expenses": [
        "Auto lease payment", 
        "Auto loan repayment", 
        "Childcare", 
        "Contribution to reserve fund", 
        "Council tax", 
        "Credit card bill", 
        "Credit card fee", 
        "Credit report", 
        "Debt collection", 
        "Drugstores and pharmacies", 
        "Education", 
        "Fuel", 
        "Funerals and bequests", 
        "Government", 
        "Groceries", 
        "Insurance", 
        "Interest", 
        "Loan repayment", 
        "Medical bill", 
        "Mortgage", 
        "Other transport", 
        "Pets", 
        "Public transport", 
        "Rent and property management fee", 
        "Retirement contributions", 
        "Ridesharing and taxis", 
        "Student loan repayment", 
        "Taxes", 
        "Utilities", 
        "Vehicle maintenance"
    ], 
    "Non-Essential Expenses": [
        "ATM/bank withdrawal", 
        "App stores", 
        "Bank fee", 
        "Books, newsletters, newspapers", 
        "Buy now, pay later", 
        "Cafes and coffee shops", 
        "Clothing", 
        "Convenience stores", 
        "Department and discount stores", 
        "Donation", 
        "Electronics", 
        "Entertainment and recreation", 
        "Firearms", 
        "Food and Drink", 
        "Gambling", 
        "Gifts", 
        "Home improvements and maintenance services", 
        "Hotels and lodging", 
        "Inter account transfer", 
        "Intra account transfer", 
        "Investment", 
        "Laundry", 
        "Legal services", 
        "Liquor", 
        "Media", 
        "Other consumer services", 
        "Other non-essential", 
        "Pawn shops", 
        "Peer to peer transfer", 
        "Recreational goods", 
        "Rent to own", 
        "SaaS tools", 
        "Self care", 
        "Sport and fitness", 
        "Toll charge", 
        "Towing companies", 
        "Trading (crypto)", 
        "Trading (non-crypto)", 
        "eCommerce purchase"
    ], 
    "Other Incoming Transactions": [
        "ATM/bank deposit", 
        "Cashback", 
        "Chargeback", 
        "Ecommerce", 
        "Grants and stipends", 
        "Insurance", 
        "Inter account transfer", 
        "Intra account transfer", 
        "Loans", 
        "Not enough information", 
        "Other", 
        "Pawn shops", 
        "Peer to peer transfer", 
        "Prenote", 
        "Refund", 
        "Reversal / adjustment", 
        "Tax refund", 
        "Trading (crypto)", 
        "Trading (non-crypto)", 
        "missing account holder information"
    ], 
    "Other Outgoing Transactions": [
        "Non-sufficient funds / Overdraft fee", 
        "Not enough information", 
        "Prenote", 
        "Reversal / adjustment", 
        "missing account holder information"
    ], 
    "Passive Income": [
        "Benefits", 
        "Interest / dividend", 
        "Property rental"
    ]
}

In [47]:
transaction_types = ', '.join(
    ['Branch TLR',
     'Cashiers Check',
     'ACH', 
     'Wire', 
     'Check', 
     'Credit Card', 
     'Transfer', 
     'Debit Card', 
     'Zelle', 
     'Bill Pay', 
     'Venmo', 
     'Apple Pay', 
     'Google Wallet', 
     'PayPal']
)

transaction_purposes = ''
for k, v in categories.items():
    transaction_purposes += ', '.join(v) + ', '
    
transaction_statement_examples = """
POS PURCHASE #2034 STARBUCKS SF CASTRO ST CA
POS PURCHASE #0765 COSTCO WHOLESALE SF
POS PURCHASE #6567 VENMO *UBER EATS 855-981-2 NY
POS CREDIT POS REFUND TERMIL 4435353 WEGMANS ITHACA NY
ACH CREDIT VANGUARD INVEST FUND DIVIDEND {name of the client}
ACH DEBIT VENMO PAYMENT SENT TO SARAH CONNOR
INCOMING WIRE REFUND - EMERSON RESORT & SPA
INCOMING WIRE ALPHABET INC CLASS A SOLIUM MORGA
DOMESTIC ONLINE WIRE FLYWIRE PAYMENTS CORPORATION 
ATM WITHDRAWAL #1734 BAY ATM LOCATOR SAN FRANCISCO CA
ATM DEPOSIT #6763 POST OFFICE SQ BOSTON MA TERMIL 3324Q
PAYROLL CREDIT ACH PACIFIC GATEWAY PAYROLL {name of the client}
ONLINE BANKING TRANSFER TO CHECKING ACCOUNT {client's account number}
ACH DEBIT CHASE AUTO LOAN BILL PAY {client's name}
ACH DEBIT PAYCHEX-HRS 401(K) {client's company name}
ACH DEBIT PERSHING BROKERAGE {client's name}
ACH DEBIT CITIZENS BANK LOAN PAYMENT {client's name}
ACH DEBIT USBANK HOME MTG PYMT {client's name}
AUTO TRANSFER TO LN TRANSFER TO SCHEDUELED LOAN PAYMENT {masked client's account numebr: XXX23443}
ACH DEBIT IRS USATAXPYMT QUARTERLY TAX 2023 {client's name}
RETURN ITEM PAYPAL INST XFFER {client's name} LINKEDIN
INTERNET TRANSFER FROM DDA {client's account number} ON 11/03 AT 08.10
MOBILE DEPOSIT
DEPOSIT TRANSFER TLR 18 BR 76 XFER FROM {client's name}
ZELLE CREDIT PAYMENT FROM: MARGARET'S BOUTIQUE L TD ID:2324424A01
ZELLE CREDIT PAYMENT FROM: TAX ARMOUT INC.
INTEREST CREDIT
DIRECT S/C WIRE TRANSFER FEE
CHECK WITHDRAWAL CK # {last four digits of checking account number}
WITHDRAWAL TLR 23 BR 90 REAL ESTATE INVESTMENT
"""

- Modular prompting

In [68]:
# Modular Prompt 1: Overview of Robert's Profile
# profile_prompt = (
#     "Robert King is a high-net-worth individual, CEO of a hedge fund in San Francisco. "
#     "He is an active philanthropist with diverse financial activities."
# )

# Modular Prompt 2: Key Banking Relationships
banking_prompt = (
    """As the CEO of Hedge Fund A, Robert King holds significant financial responsibilities and privileges with the bank. His banking relationship is characterized by the following details:
1. Authorized Signer: Robert is an authorized signer on two corporate accounts of Hedge Fund A, enabling him to conduct transactions on behalf of the company.
2. Personal Checking Account: He maintains a personal checking account primarily used for paying off his mortgage, which is worth $3 million.
3. Additional Checking Account with Debit Cards: Robert has another checking account that comes with two debit cards. 
- One card is for his personal use.
- The other card is allocated to his college-going child for day-to-day expenses.
4. Active Online Banking User: Robert frequently engages in online banking activities, utilizing platforms and services for efficient financial management.
"""
)


In [69]:
messages =  [  
{"role":"system", "content":None},    
{"role":"user", "content": None},    
]

In [78]:
message_1_content = """- Profile Summary: [{profile_prompt}]
- Banking Relationship:[{banking_prompt}]
<Your repsonse here>
""".format(profile_prompt=profile_prompt, banking_prompt=banking_prompt)

messages[1]["content"] = message_1_content

In [79]:
print(message_1_content)

- Profile Summary: [Robert King is a highly accomplished finance professional with a strong background in hedge funds. He currently serves as the CEO of Hedge Fund A in San Francisco and previously worked as a Senior Portfolio Manager at Hedge Fund B. Robert is actively involved in the community and serves as a Board Director for the San Francisco Symphony. He is estimated to have a net worth of at least $100 million, with significant liquid assets. Robert is also engaged in philanthropy and has made substantial donations to organizations such as the American Red Cross and Boys & Girls Clubs of America. He holds board positions in both corporate and nonprofit entities, including Tech Company X and the San Francisco Museum of Modern Art. In addition, Robert has been involved in successful deals as a lead partner, representing Hedge Fund A in a recent Series B investment in Tech Startup XYZ.]
- Banking Relationship:[As the CEO of Hedge Fund A, Robert King holds significant financial resp

In [83]:
# Modular Prompt 3: Transaction Generation
messages[0]["content"] = """Generate a list of transaction records for a given individual based on their profile, banking relationships, and specific criteria. 
Adhere to the following guidelines:

- Output: The transactions should be in a list with each transaction as a python dictionary representing a transaction and contain the follwoing keys with appropriate values:
'Transaction ID' (as an incrementing integer), 'Customer Key' (as an integer), 'Date', 'Account Number' (as an integer), 'Transaction Type', 'Transaction Purpose', 'Transaction Direction', 'Transaction Amount', and 'Transaction Statement'.
- Timeline: Transactions are daily, covering two months starting from 10/01/2023. 
- Use actual companies, airlines, vendors, etc., in transactions, avoiding fictitious names.
- You are allowed to generate multiple transactions on a day.
- Use the predefined transaction types, purposes, and example formats, and ensure the transactions align with the individual's financial profile and banking habits.

Predefined Transaction Types: {transaction_types}
Predefined Transaction Purposes: {transaction_categories}
Example Transaction Statements: {transaction_statement_examples}
- Profile Summary: [Profile summary will be provided by the user]
- Banking Relationship: [Banking relationship details will be provided by the user]

Note: The mimimum total number of transactions should be 10 and should not exceed 120. Aim to generate a substantial number of transactions, close to 120 if possible.""".format(transaction_types=transaction_types, transaction_categories=transaction_purposes, transaction_statement_examples=transaction_statement_examples)

In [84]:
print(messages[0]["content"])

Generate a list of transaction records for a given individual based on their profile, banking relationships, and specific criteria. 
Adhere to the following guidelines:

- Output: The transactions should be in a list with each transaction as a python dictionary representing a transaction and contain the follwoing keys with appropriate values:
'Transaction ID' (as an incrementing integer), 'Customer Key' (as an integer), 'Date', 'Account Number' (as an integer), 'Transaction Type', 'Transaction Purpose', 'Transaction Direction', 'Transaction Amount', and 'Transaction Statement'.
- Timeline: Transactions are daily, covering two months starting from 10/01/2023. 
- Use actual companies, airlines, vendors, etc., in transactions, avoiding fictitious names.
- You are allowed to generate multiple transactions on a day.
- Use the predefined transaction types, purposes, and example formats, and ensure the transactions align with the individual's financial profile and banking habits.

Predefined 

In [85]:
kwargs = {
    'verbose':False,
    'temperature':0,
    'top_p':1,
    'frequency_penalty':0,
    'presence_penalty':0
}

response = get_completion_from_messages(messages, model="gpt-3.5-turbo", **kwargs)

In [86]:
print(response)

Based on the provided profile summary and banking relationship details, here is a list of transaction records for Robert King:

1. {'Transaction ID': 1, 'Customer Key': 1, 'Date': '10/01/2023', 'Account Number': 12345678, 'Transaction Type': 'ACH', 'Transaction Purpose': 'ACH CREDIT', 'Transaction Direction': 'CREDIT', 'Transaction Amount': 50000, 'Transaction Statement': 'ACH CREDIT VANGUARD INVEST FUND DIVIDEND Robert King'}
2. {'Transaction ID': 2, 'Customer Key': 1, 'Date': '10/01/2023', 'Account Number': 12345678, 'Transaction Type': 'ACH', 'Transaction Purpose': 'ACH DEBIT', 'Transaction Direction': 'DEBIT', 'Transaction Amount': 10000, 'Transaction Statement': 'ACH DEBIT VENMO PAYMENT SENT TO SARAH CONNOR'}
3. {'Transaction ID': 3, 'Customer Key': 1, 'Date': '10/02/2023', 'Account Number': 12345678, 'Transaction Type': 'INCOMING WIRE', 'Transaction Purpose': 'INCOMING WIRE', 'Transaction Direction': 'CREDIT', 'Transaction Amount': 250000, 'Transaction Statement': 'INCOMING WIRE 

In [89]:
import tiktoken

In [90]:
def num_tokens(x, model='text-embedding-ada-002'):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(x))

In [93]:
num_tokens(response)

2442

In [94]:
num_tokens(messages[0]["content"])

1205

In [95]:
2442 + 1205

3647